# Rewriting history

Git's main job is to make sure you never lose a committed change. But it's also designed to give you total control over your development workflow. This includes letting you define exactly what your project history looks like; however, it also creates the potential of losing commits. Git provides its **history-rewriting commands** under the disclaimer that **using them may result in lost content**.



## Change the last commit (`git commit --amend`)

The `git commit --amend` command is a convenient way to **modify the most recent commit**. It lets you combine staged changes with the previous commit instead of creating an entirely new commit. It can also be used to **simply edit the previous commit message without changing its snapshot**. But, amending does not just alter the most recent commit, it **replaces it entirely**, meaning the amended commit will be a new entity with its own ref. To Git, it will look like **a brand new commit**.

With nothing in the Staging Index, running
```bash
git commit --amend -m "<new commit message>"
```
will modity the most recent commit message.

If we want to fix an error in the previous commit, **add the fix to the Staging Index**, and then
```bash
git commit --amend
```
This will prompt to add a new commit message. If **we don't want to change the commit message**, then
```bash
git commit --amend --no-edit
```

Amended commits are actually entirely new commits and the previous commit will no longer be on your current branch. This has the same consequences as resetting a public snapshot. **Avoid amending a commit that other developers have based their work on**. This is a confusing situation for developers to be in and it’s complicated to recover from.

## Change older or multiple commits (`git rebase`)

Rebase is one of two Git utilities that specializes in integrating changes from one branch onto another. The other change integration utility is `git merge`. Merge is always a forward moving change record. Alternatively, rebase has powerful history rewriting features. Rebase itself has 2 main modes: "**manual**" and "**interactive**" mode. We will cover the different Rebase modes in more detail below.

Rebasing is the process of **moving or combining a sequence of commits to a new base commit**. 

From a content perspective, rebasing is changing the base of your branch from one commit to another making it appear as if you'd created your branch from a different commit.

Internally, Git accomplishes this by creating new commits and applying them to the specified base. It's very important to understand that **even though the branch looks the same, it's composed of entirely new commits**.

The primary reason for rebasing is to maintain a linear project history. For example, consider a situation where the master branch has progressed since you started working on a feature branch. You want to get the latest updates to the master branch in your feature branch, but you want to keep your branch's history clean so it appears as if you've been working off the latest master branch. This gives the later benefit of a **clean merge** of your feature branch back into the master branch.

You have two options for integrating your feature into the master branch: **merging directly** or **rebasing and then merging**. The former option results in a **3-way merge** and a **merge commit**, while the latter results in a **fast-forward merge** and a perfectly **linear history**.

You should **never rebase commits once they've been pushed to a public repository**. The rebase would replace the old commits with new ones and it would look like that part of your project history abruptly vanished.

Suppose we are in a feature branch, running
```bash
git rebase master
```
will rebase all commits in feature onto the **local version** of master.

Sometimes we run
```bash
git rebase master feature2
```
and it is equivalent to
```bash
git checkout feature2
git rebase master
```

It is recommended to run rebase with `-i` or `--interactive`. This will prompt up the history of commits that are going to be rebased in a text editor, and we can choose whether to **pick** a commit or to **squash** it, meaning that commit will be absorbed into the next commit. This is how `git rebase` makes the commit history cleaner.

In complicated cases like `next` branched off from `master`, and we have a `feature` based on branch `next`, running
```bash
git rebase --onto master next feature
```
will rebase `feature` onto `master` such that it appears to be made on top of `master` from the beginning.

One caveat to consider when working with Git Rebase is **merge conflicts** may become more frequent during a rebase workflow. This occurs if you have a long-lived branch that has strayed from master. Eventually you will want to rebase against master and at that time it may contain many new commits that your branch changes may conflict with. This is easily remedied by rebasing your branch frequently against master, and making more frequent commits. The `--continue` and `--abort` command line arguments can be passed to git rebase to advance or reset the the rebase when dealing with conflicts.

A more serious rebase caveat is **lost commits from interactive history rewriting**. Running rebase in interactive mode and executing subcommands like squash or drop will remove commits from your branche's immediate log. At first glance this can appear as though the commits are permanently gone. Using git reflog these commits can be restored and the entire rebase can be undone.

Git Rebase itself is not seriously dangerous. The real danger cases arise when **executing history rewriting interactive rebases and force pushing the results to a remote branch that's shared by other users**. This is a pattern that should be avoided as it has the capability to overwrite other remote users' work when they pull.

## `git reflog`

Unlike `git log` which tracks the **commit history** of the repo, `git reflog` track when **Git refs** were updated in the local repository. Common examples include:
+ `git checkout`
+ `git reset`
+ `git merge`

In addition to **branch tip reflogs**, a special reflog is maintained for the **Git stash**. Reflogs are stored in directories under the local repository's `.git` directory.

Running
```bash
git reflog`
```
is equivalent to run
```bash
git reflog show HEAD
```

Other common usages:
```bash
git reflog show --all
```
shows a complete reflog of **all refs**.

```bash
git reflog show <branch>
```
shows reflog for a **branch**.

```bash
git reflog stash
```
shows reflog for the **stash cache**.

When checking history, it is possible to add **time qualifiers** to the branch name, i.e. `branch@{<qualifier>}`, such as
+ 1.minute.ago
+ 1.hour.ago
+ 1.day.ago
+ yesterday
+ 1.week.ago
+ 1.month.ago
+ 1.year.ago
+ 2011-05-17.09:00:00

So, it is possible to check reflog like
```bash
git reflog show master@{yesterday}
```

Another subcommand
```bash
git reflog expire
```
**cleans up** old or unreachable reflog entries.

And
```bash
git reflog delete <ref>
```
will **delete** a passed in reflog entry.